# Main Jupyter Notebook for WULPUS

Sergei Vostrikov, Cédric Hirschi

ETH Zurich, 2024

---

If you are not familiar with Jupyter Notebooks, please first check online tutorials such as https://realpython.com/jupyter-notebook-introduction/#creating-a-notebook

## Preparation

First, we import the libraries we will use next to the WULPUS library.

In [1]:
%matplotlib widget
import ipywidgets as wdg
import matplotlib.pyplot as plt
import numpy as np

Here, we test if the `matplotlib widget` backend is working. If you do not see an interactive plot after executing below cell, try:
- Restart the kernel
- Manually install `ipympl` via `conda install -c conda-forge ipympl` inside your `wulpus_env` environment 

In [ ]:
plt.figure()
plt.plot([[1, 2], [0, 0]])
plt.show()

## Configuration

### TX/RX Channel Configuration

You can set up the TX/RX channel configuration either via

- The dedicated GUI
- The Python API directly
- By loading a configuration file

Each of the following cells demonstrates one of these methods.

In [ ]:
# Configure the TX/RX channel configuration using the GUI
from wulpus.trx_conf.gui import WulpusTRXConfigGUI 

conf_gen = WulpusTRXConfigGUI()

display(conf_gen)

In [5]:
# Configure the TX/RX channel configuration using the API directly
# UNCOMMENT THE CODE IN THIS CELL WHEN YOU ARE NOT USING THE GUI
# from wulpus.trx_conf.gen import WulpusTRXConfigGen

# # TX and RX active channels IDs
# conf_gen = WulpusTRXConfigGen()

# # The first array is a set of TX chanels, the second one - RX.
# conf_gen.add_config([7], [7], 
#                     optimized_switching=False)
# conf_gen.add_config([0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7], 
#                     optimized_switching=True)

In [7]:
# Configure the TX/RX channel configuration by loading them from a file
# UNCOMMENT THE CODE IN THIS CELL WHEN YOU ARE NOT USING THE GUI OR THE API DIRECTLY
# from wulpus.trx_conf.gui import WulpusTRXConfigGUI

# conf_gen = conf_gui.WulpusRxTxConfigGenGUI()

# conf_gen.with_file("tx_rx_configs.json")

### Ultrasound Subsystem (USS) Configuration

Similar to the TX/RX channel configuration, you can set up the USS configuration either via

- The dedicated GUI
- The Python API directly
- By loading a configuration file

Here, we always set up a configuration using the API first (optionally using a configuration file) and then pass it to the GUI.

For this, you must have the TX/RX channel configuration set up first as `conf_gen`.

In [ ]:
from wulpus.config_package import PGA_GAIN
from wulpus.uss_conf.gen import WulpusUSSConfigGen
from wulpus.uss_conf.gui import WulpusUSSConfigGUI

# Get TX/RX configuration
tx_confs = conf_gen.get_tx_configs()
rx_confs = conf_gen.get_rx_configs()

# Create USS configuration using the API
uss_conf = WulpusUSSConfigGen(num_acqs=1000,
                           dcdc_turnon=100,
                           start_hvmuxrx=500,
                           meas_period=20000,
                           num_txrx_configs=len(tx_confs),
                           tx_configs=tx_confs,
                           rx_configs=rx_confs,
                           rx_gain=PGA_GAIN[19],
                           pulse_freq=1000000,
                           num_pulses=10,
                           sampling_freq=4000000)

# Optionally, modify the configuration using the GUI
uss_conf = WulpusUSSConfigGUI(uss_conf)

# or load the configuration from a file
# uss_conf.with_file('uss_config.json')

display(uss_conf)

## Connect

We initialize a `WulpusConnection` object here such that it can run independently from the main GUI.

The connection can be of different types:
- nRF52 Dongle which is paired with the probe (via `'dongle'`, default)
- Direct BLE connection (via `'direct'`) 

In [5]:
from wulpus.connection.connection import WulpusConnection

# Open a connection
connection = WulpusConnection('direct') # direct connection via dongle
# connection = WulpusConnection() # connection via serial port and dongle

# # Optionally, get the available devices
# connection.get_available()

## Make Measurement

Finally, we can make a measurement using the main GUI. Check the WULPUS user manual (chapter 3 and 4) for more information on how to use the GUI.

In [ ]:
from wulpus.gui import WulpusGuiSingleCh

# If the GUI is already open and the port is opened, close it
if 'gui' in globals():
    if gui.port_opened:
        gui.com_link.close()

# Create the GUI
gui = WulpusGuiSingleCh(connection, uss_conf, max_vis_fps=20)
display(gui)

## Loading and interpreting the saved data

The data is saved in a `.npz` file. This is a compressed file format that can be used with the numpy library.

Here, we go through the process of loading the data and plotting it.

In [ ]:
# Load the data
data = np.load('examples/data_0.npz')

print('Keys:', data.files)

The structure is a dictionary with the length `num_acquisitions` (Number of acquisitions) and consists of the following keys:

**data_arr:** This column actually contains the data, each of length `num_samples` (Number of samples). This can be seen as data for one acquisition.

**acq_num_arr:** The sequential number for each acquisition. MSP430 increments this number by 1 every new acquisition, starting from zero. If one number is missing (a step is larger than one), then this acquisition got dropped during the BLE transmission.

**tx_rx_id_arr:** The TX/RX configuration ID for the acquisition. This ids  are the same as the ones in the `Active RX ID` dropdown menu in the GUI (the same as Config field in the RX/TX config GUI).

In [ ]:
print('Data shape:', data['data_arr'].shape)
print('Unique IDs:', np.unique(data['tx_rx_id_arr']))

In `examples/example_0.npz`, we have 100 acquisitions, each containing 400 samples. The WULPUS system was connected to the 8 channels of the linear array transducer (2.25 MHz), placed on the forearm, while the proband was performing periodic hand gestures. There was only one TX/RX configuration programmed (config id = 0). With this configuration, 8 channel were used at both transmit and receive. The data was acquired with the settings saved in the files `./examples/tx_rx_configs.json` and `./examples/uss_config.json`.

Since the `data_arr` is a bit awkward to use in this form (`(400, 100)`), we transpose it to `(100, 400)`.

In [ ]:
# Transpose the data such that we can index via time
data_meas = data['data_arr'].T

print('Data shape:', data_meas.shape)

The data can then be easily used further, for example to just make a plot of one acquisition. 

As a general example, we show how to select the data only for the TX/RX config with ID **n = 0**.

In [ ]:
channel_id = 0

data_sel = data_meas[data['tx_rx_id_arr'] == channel_id, :]

print('Data shape:', data_sel.shape)


We visualize the data with `matplotlib` and `ipywidgets`.
By interacting with the slider, a user can see the data over time.

In [ ]:
plt.figure(figsize=(10, 5))
plot_data = plt.plot(data_sel[0], linestyle='-', marker='o', linewidth=1, markersize=2)
plt.ylim(-2500, 2500)
plt.title(f'Acquisition 0/{data_sel.shape[0]}')
plt.xlabel('Samples')
plt.ylabel('ADC digital code')
plt.grid()
plt.show()

def visualize(frame):
    plot_data[0].set_ydata(data_sel[frame])
    plt.title(f'Acquisition {frame}/{data_sel.shape[0]}')
    plt.draw()

wdg.interact(visualize, frame=wdg.IntSlider(min=0, max=data_sel.shape[0]-1, step=1, value=10))

With this data, you can now perform further analysis, such as filtering, envelope extraction, etc.

---
```text
   Copyright (C) 2024 ETH Zurich. All rights reserved.
   Author: Sergei Vostrikov, ETH Zurich
           Cedric Hirschi, ETH Zurich
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

   SPDX-License-Identifier: Apache-2.0
```